In [2]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt

In [10]:
IMAGE_SIZE = 128
def getDataset(path):
    return tf.keras.preprocessing.image_dataset_from_directory(
        path,
        labels='inferred',
        image_size=(IMAGE_SIZE, IMAGE_SIZE),
        color_mode='grayscale',
    )

# Construct the Columbia datasets
def invert(x,y): return (-x, y)

data = getDataset('../data/created_dataset')
data_inv = data.map(invert)

Found 7200 files belonging to 24 classes.


In [19]:
X = np.empty((14400, 128, 128, 1))
y = np.empty(14400)

i = 0
for batch, lbls in data.take(-1):
    X[i*32:(i+1)*32] = batch
    y[i*32:(i+1)*32] = 0
    i += 1

for batch, lbls in data_inv.take(-1):
    X[i*32:(i+1)*32] = batch
    y[i*32:(i+1)*32] = 1
    i += 1

In [42]:
l = [
    layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1, input_shape=(128,128,1)),
    layers.Flatten(),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),

    layers.Dense(10, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    
    layers.Dense(1, activation='sigmoid')
]

model = tf.keras.Sequential(l)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_4 (Rescaling)      (None, 128, 128, 1)       0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               2097280   
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 128)               512       
_________________________________________________________________
dense_25 (Dense)             (None, 10)                1290      
_________________________________________________________________
dropout_20 (Dropout)         (None, 10)               

In [43]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-2),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.evaluate(X, y, steps=5)
history = model.fit(X, y, validation_split=0.2, epochs=5)

5/5 [==============================] - 2s 317ms/step - loss: 0.8727 - accuracy: 0.3355
Epoch 1/5
360/360 [==============================] - 10s 29ms/step - loss: 0.0714 - accuracy: 0.9778 - val_loss: 6.3300e-06 - val_accuracy: 1.0000
Epoch 2/5
360/360 [==============================] - 11s 30ms/step - loss: 0.0405 - accuracy: 0.9825 - val_loss: 5.8913e-06 - val_accuracy: 1.0000
Epoch 3/5
360/360 [==============================] - 13s 36ms/step - loss: 0.0449 - accuracy: 0.9802 - val_loss: 1.1276e-05 - val_accuracy: 1.0000
Epoch 4/5
360/360 [==============================] - 12s 33ms/step - loss: 0.0382 - accuracy: 0.9835 - val_loss: 3.6845e-04 - val_accuracy: 1.0000
Epoch 5/5
360/360 [==============================] - 12s 34ms/step - loss: 0.0401 - accuracy: 0.9838 - val_loss: 3.7038e-06 - val_accuracy: 1.0000


In [45]:
tf.keras.models.save_model(model, 'inverted.h5')